In [65]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.autograd import Variable
from dataset import AirplaneDataset
from matplotlib.pyplot import imshow
import numpy as np
%matplotlib inline

In [66]:
# !unzip "/content/Retail Pulse ML Assignment Data.zip"

In [67]:
# set datapath to the 'Retail Pulse ML Assignment' folder
DATAPATH = '/content/'

train_dataset, validate_dataset = AirplaneDataset(phase='train'), AirplaneDataset(phase='val')

train_loader, validate_loader = DataLoader(train_dataset, batch_size=100, shuffle=True), DataLoader(validate_dataset, batch_size=100)

num_classes = train_dataset.num_classes

In [45]:
class CNNModel(nn.Module):
  '''
  Model class with 2 convolutional layers 
  and 2 fully connected layers
  '''

    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        # Fully Connected
        self.dropout1 = nn.Dropout()
        self.fc1 = nn.Linear(32 * 18 * 18, 4096)
        self.relu4 = nn.ReLU()
        self.dropout2 = nn.Dropout()
        self.fc2 = nn.Linear(4096, num_classes)
    

    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out) 
        # Max pool 1
        out = self.maxpool1(out)
        
        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)
        # Max pool 2
        out = self.maxpool2(out)
        
        out = out.view(out.size(0), -1)
        
        # Linear
        out = self.dropout1(out)
        out = self.fc1(out)
        out = self.relu4(out)
        out = self.dropout2(out)
        out = self.fc2(out)
        
        return out

In [69]:
# Prepare the model, loss, optimizer
model = CNNModel()
if torch.cuda.is_available():
  model.cuda()
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [68]:
# Verify the parameter shapes
print(model.parameters())
print(len(list(model.parameters())))

for i in range(len(list(model.parameters()))):
  print(list(model.parameters())[i].size())

<generator object Module.parameters at 0x7fd7faf75e60>
8
torch.Size([16, 3, 3, 3])
torch.Size([16])
torch.Size([32, 16, 3, 3])
torch.Size([32])
torch.Size([4096, 10368])
torch.Size([4096])
torch.Size([70, 4096])
torch.Size([70])


In [ ]:
# Set the iterations, epochs
n_iters = 3000
batch_size = 100
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

In [71]:
iter = 0
    
for epoch in range(num_epochs):
    for i, (images,labels) in enumerate(train_loader):

        if torch.cuda.is_available():
          images = Variable(images.cuda())
          labels = Variable(labels.cuda())
        else:
          images = Variable(images)
          labels = Variable(labels)
        
        optimizer.zero_grad()
        
        model.train()
        outputs = model(images)

        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()

        iter += 1
        
        if iter%100 == 0:
            correct = 0
            total = 0
            for images, labels in validate_loader:
                if torch.cuda.is_available():
                  images = Variable(images.cuda())
                else:
                  images = Variable(images)
                
                model.eval()
                outputs = model(images)
                
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                  correct += (predicted==labels).sum()

            accuracy = 100 * correct / total
            
            print("Iteration: {}. Loss {}. Accuracy: {}".format(iter, loss.item(), accuracy))
            

Iteration: 100. Loss 3.709230422973633. Accuracy: 9
Iteration: 200. Loss 3.1367685794830322. Accuracy: 12
Iteration: 300. Loss 2.4731619358062744. Accuracy: 21
Iteration: 400. Loss 1.8575758934020996. Accuracy: 26
Iteration: 500. Loss 0.7299532294273376. Accuracy: 30
Iteration: 600. Loss 1.1300098896026611. Accuracy: 31
Iteration: 700. Loss 0.21012969315052032. Accuracy: 36
Iteration: 800. Loss 0.10335109382867813. Accuracy: 36
Iteration: 900. Loss 0.10332939773797989. Accuracy: 36
Iteration: 1000. Loss 0.06652893871068954. Accuracy: 37
Iteration: 1100. Loss 0.02278219163417816. Accuracy: 38
Iteration: 1200. Loss 0.0652838796377182. Accuracy: 38
Iteration: 1300. Loss 0.014399146661162376. Accuracy: 38
Iteration: 1400. Loss 0.006726131308823824. Accuracy: 39
Iteration: 1500. Loss 0.0043958090245723724. Accuracy: 39
Iteration: 1600. Loss 0.005041284486651421. Accuracy: 39
Iteration: 1700. Loss 0.037220925092697144. Accuracy: 37
Iteration: 1800. Loss 0.0030189515091478825. Accuracy: 39
It

In [76]:
# Post training calculate total accuracy, classwise accuracies
classwise_accuracy = {}

for i in range(70):
  classwise_accuracy[i] = (0,0)

for images, labels in validate_loader:

    if torch.cuda.is_available():
      images = Variable(images.cuda())
    else:
      images = Variable(images)
    
    model.eval()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)

    labels_list = labels.tolist()
    pred_list = predicted.tolist()

    for i in range(predicted.size(0)):
      (x,y) = classwise_accuracy[labels_list[i]]
      if (pred_list[i]==labels_list[i]):
        x = x + 1
      y = y+1
      classwise_accuracy[labels_list[i]] = (x,y)
    
    if torch.cuda.is_available():
        correct += (predicted.cpu() == labels.cpu()).sum()
    else:
      correct += (predicted==labels).sum()

accuracy = 100 * correct / total

print("Final: Accuracy: {}\n".format(accuracy))

# print classwise accuracies with class name
with open(os.path.join(DATAPATH,'families.txt'),'r') as f:
    for idx,line in enumerate(f.readlines()):
        (x,y) = classwise_accuracy[idx]
        print("Class :{}, Class name :{}, Accuracy :{}".format(idx,line.strip(),100 * x/y))


Final: Accuracy: 39

Class :0 Class name :A300 Accuracy :18.181818181818183
Class :1 Class name :A310 Accuracy :11.764705882352942
Class :2 Class name :A320 Accuracy :42.857142857142854
Class :3 Class name :A330 Accuracy :28.35820895522388
Class :4 Class name :A340 Accuracy :57.89473684210526
Class :5 Class name :A380 Accuracy :27.272727272727273
Class :6 Class name :ATR-42 Accuracy :17.647058823529413
Class :7 Class name :ATR-72 Accuracy :39.39393939393939
Class :8 Class name :An-12 Accuracy :51.515151515151516
Class :9 Class name :BAE 146 Accuracy :43.28358208955224
Class :10 Class name :BAE-125 Accuracy :33.333333333333336
Class :11 Class name :Beechcraft 1900 Accuracy :23.529411764705884
Class :12 Class name :Boeing 707 Accuracy :12.121212121212121
Class :13 Class name :Boeing 717 Accuracy :12.121212121212121
Class :14 Class name :Boeing 727 Accuracy :17.647058823529413
Class :15 Class name :Boeing 737 Accuracy :60.150375939849624
Class :16 Class name :Boeing 747 Accuracy :48.50746

In [77]:
# Save the model
torch.save(model.state_dict(), 'best_model.pt')